In [4]:
!pip install fastai2

     |████████████████████████████████| 184kB 8.7MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 


In [5]:
!pip install fastcore==0.1.35

  Found existing installation: fastcore 1.3.19
    Uninstalling fastcore-1.3.19:
      Successfully uninstalled fastcore-1.3.19


In [6]:
!pip install torch==1.6.0 torchvision==0.7.0

     |████████████████████████████████| 748.8MB 22kB/s 
     |████████████████████████████████| 5.9MB 41.1MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [7]:
from fastai2.text.all import *

In [8]:
path = untar_data(URLs.IMDB)

In [9]:
path.ls()

(#7) [Path('/root/.fastai/data/imdb/README'),Path('/root/.fastai/data/imdb/tmp_clas'),Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/imdb.vocab'),Path('/root/.fastai/data/imdb/unsup'),Path('/root/.fastai/data/imdb/test'),Path('/root/.fastai/data/imdb/tmp_lm')]

In [10]:
dls = TextDataLoaders.from_folder(path, valid='test', bs=32) # our validation data is in the 'test' directory instead of 'valid' (default)

In [11]:
dls.show_batch()

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad,neg
2,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad,pos
3,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad,pos
4,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad 

In [12]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [13]:
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.607362,0.398740,0.821720,04:06


epoch,train_loss,valid_loss,accuracy,time
0,0.320568,0.254508,0.896560,08:07
1,0.235238,0.195447,0.922600,08:07
2,0.195127,0.199295,0.923760,08:12
3,0.164513,0.195095,0.927400,08:11


In [14]:
learn.show_results()

,text,category,category_
0,"xxbos xxmaj there 's a sign on xxmaj the xxmaj lost xxmaj highway that says : \n\n * major xxup spoilers xxup ahead * \n\n ( but you already knew that , did n't you ? ) \n\n xxmaj since there 's a great deal of people that apparently did not get the point of this movie , xxmaj i 'd like to contribute my interpretation of why the plot makes perfect sense . xxmaj as others have pointed out , one single viewing of this movie is not sufficient . xxmaj if you have the xxup dvd of xxup md , you can "" cheat "" by looking at xxmaj david xxmaj lynch 's "" top 10 xxmaj hints to xxmaj unlocking xxup md "" ( but only upon second or third viewing , please . ) ;) \n\n xxmaj first of all , xxmaj mulholland xxmaj drive is",pos,pos
1,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad,pos,pos
2,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad,pos,pos
3,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad,neg,neg
4,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpa

In [15]:
learn.predict('I really love this movie! :)')

('pos', tensor(1), tensor([6.0749e-05, 9.9994e-01]))

In [17]:
learn.predict("Don't even bother watching this movie")

('neg', tensor(0), tensor([0.8633, 0.1367]))

## Using Data-block API

In [18]:
imdb = DataBlock(blocks=(TextBlock.from_folder(path), CategoryBlock),
                 get_items=get_text_files,
                 get_y=parent_label,
                 splitter=GrandparentSplitter(valid_name='test'))

In [19]:
dls = imdb.dataloaders(path)

## The ULMFit Approach

Previously, we used a pretrained language model from Wikipedia and directly fine-tuned it to a movie review classifier. Now, we fine-tune our pretrained language model to the IMDB corpus and then use it for our base classifier. This will result in a language model that is particularly good at predicting the next word of a movie review. In contrast, the pretrained model was trained only on Wikipedia articles.
<img src='https://docs.fast.ai/images/ulmfit.png'>

In [20]:
dls_lm = TextDataLoaders.from_folder(path, is_lm=True, valid_pct=0.1)

In [21]:
dls_lm.show_batch()

,text,text_
0,"xxbos i really love action / adventure films and this is one of the best . xxmaj of course , we love the stars of this genre , xxmaj bruce xxmaj willis , xxmaj sly xxmaj stallone , xxmaj clint xxmaj eastwood , xxmaj kurt xxmaj russell , xxmaj arnold xxmaj schwarzenegger and the rest , but its the story that really counts . xxmaj the best are somewhat feasible but we","i really love action / adventure films and this is one of the best . xxmaj of course , we love the stars of this genre , xxmaj bruce xxmaj willis , xxmaj sly xxmaj stallone , xxmaj clint xxmaj eastwood , xxmaj kurt xxmaj russell , xxmaj arnold xxmaj schwarzenegger and the rest , but its the story that really counts . xxmaj the best are somewhat feasible but we can"
1,"violence , a high body count , some wonderful countryside vistas , and a story with a strong forward momentum . xxmaj nazis always make for wonderfully nasty , vile baddies one can have a lot of fun booing , and the ones here are no different . xxmaj francesco xxmaj de xxmaj masi contributes an excellent music score . xxmaj the film gets off to a good start with some striking",", a high body count , some wonderful countryside vistas , and a story with a strong forward momentum . xxmaj nazis always make for wonderfully nasty , vile baddies one can have a lot of fun booing , and the ones here are no different . xxmaj francesco xxmaj de xxmaj masi contributes an excellent music score . xxmaj the film gets off to a good start with some striking visuals"
2,"cliche - ridden as it may be , this comedy works , and the reason can be summed up in two words : xxmaj jack xxmaj lemmon . xxmaj the whole concept of the story is tried and true , but tired ; and this is definitely not one of xxmaj simon 's best efforts ( good as it is ) . xxmaj but xxmaj lemmon immerses himself in his character ,","- ridden as it may be , this comedy works , and the reason can be summed up in two words : xxmaj jack xxmaj lemmon . xxmaj the whole concept of the story is tried and true , but tired ; and this is definitely not one of xxmaj simon 's best efforts ( good as it is ) . xxmaj but xxmaj lemmon immerses himself in his character , takes"
3,"in a vacuum of characters . when someone crushes a milk carton you do n't grieve the milk carton unless you can xxunk ( xxunk ) it in some way and these characters were n't ; they were empty . xxmaj there was no conflict . the sole redeeming quality i enjoyed about the film is the way it removed the veneer of eroticism and beauty from violence , and made it","a vacuum of characters . when someone crushes a milk carton you do n't grieve the milk carton unless you can xxunk ( xxunk ) it in some way and these characters were n't ; they were empty . xxmaj there was no conflict . the sole redeeming quality i enjoyed about the film is the way it removed the veneer of eroticism and beauty from violence , and made it trite"
4,"really was no hope . xxmaj the story was flimsier than rice paper and the special effects shots were mostly stolen , many from xxup battle xxup beyond xxup the xxup stars . xxmaj the ship was apparently made of card board and the technology was primitive enough that humans who had never seen the systems could take control and hack the computer . xxmaj pathetic . xxmaj i 've seen fan","was no hope . xxmaj the story was flimsier than rice paper and the special effects shots were mostly stolen , many from xxup battle xxup beyond xxup the xxup stars . xxmaj the ship was apparently made of card board and the technology was primitive enough that humans who had never seen the systems could take control and hack the computer . xxmaj pathetic . xxmaj i 've seen fan -"
5,"spirit lives by the recent army - generated antagonism by xxmaj red xxmaj cloud . \n\n xxmaj finally , we can also compare this story with xxmaj anthony xxmaj mann 's later film "" the xxmaj far xxmaj country "" , starring xxmaj jimmy xxmaj stewart . xxmaj in xxmaj stewart 's case , he can choose to retu

In [22]:
# apply the same AWD LSTM model with additional metrics 'Perplexity' (exponential of loss) and weight decay
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16() # .to_fp16() puts Learner in mixed precision for speeding up training process in GPU

In [23]:
# by default, pretrained Learner is frozen, meaning that only the head will train while the body stays frozen.
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.536295,4.122207,0.285915,61.695232,22:05


In [24]:
# save the results
learn.save('1epoch')

In [27]:
# we can load the saved model
learn = learn.load('1epoch')

In [28]:
# fine-tune the model
learn.unfreeze() # don't forget to unfreeze the model
learn.fit_one_cycle(3, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.248290,3.946480,0.307094,51.752853,23:55
1,4.173913,3.865825,0.316095,47.742664,23:54
2,4.113593,3.824888,0.321229,45.827690,23:51


In [29]:
# save_encoder will only save the upper layers (except the final classifier layer)
learn.save_encoder('finetuned')

Before diving into training a classifier, let's test our model by creating a random review

In [30]:
# generate random reviews
TEXT = 'I like this movie because'
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)]

In [31]:
print("\n".join(preds))

i like this movie because it was so bad that i even did not make it to the end . It was so bad . What was it with the casting ? The actors were the only funny things . You
i like this movie because there are no sources for it , so I 'm glad i did n't spoil it . Its a nice movie . i was totally surprised that it was not shown on TV . i thought it


## Training the Text Classifier
Now, let's train the classifier part

In [33]:
dls_clf = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test', text_vocab=dls_lm.vocab)

In [34]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=.5, metrics=accuracy)

In [35]:
# train by gradually unfreezing a few layers at a time
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.590103,0.369675,0.837440,03:31


In [36]:
learn.freeze_to(-2) # freeze all, except the last two parameter groups
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.345752,0.276797,0.886440,03:58


In [37]:
# unfreeze a bit more..
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.261518,0.224264,0.911480,05:30


In [38]:
# finally, unfreeze the whole model
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.213985,0.201439,0.922800,06:56
1,0.194253,0.198155,0.924680,07:04
